In [28]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import dask.dataframe as dd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import itertools
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from catboost import CatBoostClassifier, Pool
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from scipy.special import boxcox1p
from scipy.special import inv_boxcox1p
import sklearn

In [2]:
!pip install openpyxl


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [29]:
def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def data_by_bins(y_test, y_preds, bins=20):   
    y_test = pd.Series(y_test)
    y_preds = pd.Series(y_preds)  
    
    intervals=list()
    intervals.append(-np.inf)   
    
    print("Computing intervals")
    for i in tqdm(range(1, bins)):
        intervals.append(y_preds.quantile(i / bins))        
    intervals.append(np.inf)
    
    interval_data=list()
    print("Collecting intervals data")    
    for i in tqdm(range(len(intervals)-1)):       
        y_preds_interval_index = y_preds[(y_preds >= intervals[i]) & (y_preds <= intervals[i+1])].index 
        interval_data.append(np.array(y_test[y_preds_interval_index]))  
        
    return interval_data,intervals

In [4]:
messages=pd.read_excel("data/Датасеты/messages.xlsx", header=0)
test_intervals=pd.read_excel("data/Датасеты/test_intervals.xlsx", header=0)
messages.ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ=pd.to_datetime(messages.ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ)

In [5]:
y_fact_test=pd.DataFrame()
y_pred_test=pd.DataFrame()
for i in tqdm(range(4,10)):
    y_p=pd.read_csv(f"pred_data/y_test_exg_{i}.csv")
    y_fact_test=pd.concat([y_fact_test,y_p],axis=0)
    del y_p
    y_t=pd.read_csv(f"pred_data/y_preds_exg_{i}.csv")
    y_pred_test=pd.concat([y_pred_test,y_t],axis=0)
    del y_t
    
y_pred_test["0"]=boxcox1p(inv_boxcox1p(y_pred_test["0"], 0.7)/60, 0.7)
y_fact_test.y=boxcox1p(inv_boxcox1p(y_fact_test.y, 0.7)/60, 0.7)

  0%|          | 0/6 [00:00<?, ?it/s]

In [6]:
interval_data, intervals=data_by_bins(y_fact_test.y, y_pred_test["0"], bins=20) 

Computing intervals


  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [7]:
def make_intervals_with_M1(msg):
    return msg.ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ.tolist()

def make_intervals_without_M1(y_test,msg,delta):
    count=0
    t_list=list()
    for i in tqdm(range(int(y_test.DT.min().timestamp())+delta,int(y_test.DT.max().timestamp()),int(delta))):
        t1=i
        t2=i+delta
        for j in range(msg.shape[0]):
            was_M1=0
            if (t1>msg.ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ.iloc[j].timestamp()) & (t1<msg.ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ.iloc[j].timestamp()):
                was_M1=1
                break
            if (t2>msg.ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ.iloc[j].timestamp()) & (t2<msg.ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ.iloc[j].timestamp()):
                was_M1=1
                break              
            if (t1<msg.ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ.iloc[j].timestamp()) & (t2>msg.ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ.iloc[j].timestamp()):
                was_M1=1
                break    
        if was_M1==0:
            t_list.append(t1)
        count+=was_M1
    #print(count)
    return pd.to_datetime(t_list,unit='s')

In [8]:
def find_prob(model_prediction, horizon, intervals, interval_data):    
    for i in range(len(intervals)):
        if model_prediction < intervals[i]:              
            break       
    return (interval_data[i-1] <= horizon).sum() / interval_data[i-1].shape[0]

In [9]:
def calculate_prob_by_interval(interv,y_test,y_preds,interval_data,intervals):
    probs=list()
    
    print("Computing probabilities")   
    for i in tqdm(range(len(interv))):
        t=interv[i]-d_t
        nearest_t=find_nearest(y_test.DT, t)
        w=y_test[y_test.DT==nearest_t]
        prediction=y_preds["0"][w.index].iloc[0]
        prediction_to_minutes=boxcox1p(inv_boxcox1p(prediction, 0.7)/60, 0.7)
                
        p=find_prob(prediction_to_minutes, horizon, intervals, interval_data) 
        #print(p,prediction_to_minutes)  
        probs.append(p)
    return probs

In [10]:
def prepare_data(exg_number, take_first_elements = 10):
    y_preds=pd.read_csv(f"pred_data/y_preds_exg_{exg_number}.csv")
    y_test=pd.read_csv(f"pred_data/y_test_exg_{exg_number}.csv")
    y_test.DT=pd.to_datetime(y_test.DT)
    
    msg=messages[(messages.ВИД_СООБЩЕНИЯ=="M1") & (messages.ИМЯ_МАШИНЫ==f"ЭКСГАУСТЕР А/М №{exg_number}")]
    #msg=msg[(msg.ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ>y_test.DT.min()) & (msg.ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ<y_test.DT.max())]
    msg=msg[(msg.ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ>pd.to_datetime('2020-09-01 00:00:00')) & (msg.ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ<pd.to_datetime('2021-12-31 23:59:59'))]    
    
    intervals_with_M1=make_intervals_with_M1(msg)
    intervals_without_M1=make_intervals_without_M1(y_test,msg,delta)
     
    p_M1_YES=calculate_prob_by_interval(intervals_with_M1[:take_first_elements],y_test,y_preds,interval_data,intervals)
    p_M1_NO=calculate_prob_by_interval(intervals_without_M1[:take_first_elements],y_test,y_preds,interval_data,intervals)
    return p_M1_YES, p_M1_NO

In [11]:
def calculate_metrcis(thresholder, p_M1_YES, p_M1_NO):   
    res_pred=np.concatenate([p_M1_YES,p_M1_NO])
    res_predictions=(res_pred>thresholder).astype(int)
    res_fact=np.concatenate([np.ones(len(p_M1_YES)),np.zeros(len(p_M1_NO))])
    accuracy=sklearn.metrics.accuracy_score(res_fact, res_predictions)
    precision=sklearn.metrics.precision_score(res_fact, res_predictions)
    recall=sklearn.metrics.recall_score(res_fact, res_predictions)
    f1_score=sklearn.metrics.f1_score(res_fact, res_predictions)
    return accuracy,precision,recall,f1_score

In [12]:
delta=(test_intervals.finish-test_intervals.start).mean().seconds
delta_minutes=delta/60
d_t=datetime.timedelta(seconds=delta_minutes/2)
horizon= boxcox1p(delta_minutes, 0.7)

In [13]:
p_M1_YES_exg_4, p_M1_NO_exg_4 = prepare_data(4, take_first_elements = 100)

  0%|          | 0/816 [00:00<?, ?it/s]

Computing probabilities


  0%|          | 0/8 [00:00<?, ?it/s]

Computing probabilities


  0%|          | 0/100 [00:00<?, ?it/s]

In [14]:
calculate_metrcis(0.0048, p_M1_YES_exg_4, p_M1_NO_exg_4)

(0.9074074074074074, 0.4375, 0.875, 0.5833333333333334)

In [15]:
p_M1_YES_exg_5, p_M1_NO_exg_5 = prepare_data(5, take_first_elements = 100)

  0%|          | 0/2296 [00:00<?, ?it/s]

Computing probabilities


  0%|          | 0/16 [00:00<?, ?it/s]

Computing probabilities


  0%|          | 0/100 [00:00<?, ?it/s]

In [31]:
calculate_metrcis(0.005, p_M1_YES_exg_5, p_M1_NO_exg_5)

(0.7586206896551724, 0.125, 0.125, 0.125)

In [17]:
p_M1_YES_exg_6, p_M1_NO_exg_6 = prepare_data(6, take_first_elements = 100)

  0%|          | 0/1104 [00:00<?, ?it/s]

Computing probabilities


  0%|          | 0/4 [00:00<?, ?it/s]

Computing probabilities


  0%|          | 0/100 [00:00<?, ?it/s]

In [32]:
calculate_metrcis(0.0038, p_M1_YES_exg_6, p_M1_NO_exg_6)

(0.8076923076923077, 0.1, 0.5, 0.16666666666666669)

In [19]:
p_M1_YES_exg_7, p_M1_NO_exg_7 = prepare_data(7, take_first_elements = 100)

  0%|          | 0/1025 [00:00<?, ?it/s]

Computing probabilities


  0%|          | 0/9 [00:00<?, ?it/s]

Computing probabilities


  0%|          | 0/100 [00:00<?, ?it/s]

In [20]:
calculate_metrcis(0.0053, p_M1_YES_exg_7, p_M1_NO_exg_7)

(0.9541284403669725, 1.0, 0.4444444444444444, 0.6153846153846153)

In [21]:
p_M1_YES_exg_8, p_M1_NO_exg_8 = prepare_data(8, take_first_elements = 100)

  0%|          | 0/2074 [00:00<?, ?it/s]

Computing probabilities


  0%|          | 0/4 [00:00<?, ?it/s]

Computing probabilities


  0%|          | 0/100 [00:00<?, ?it/s]

In [33]:
calculate_metrcis(0.0053, p_M1_YES_exg_8, p_M1_NO_exg_8)

(0.9807692307692307, 1.0, 0.5, 0.6666666666666666)

In [23]:
p_M1_YES_exg_9, p_M1_NO_exg_9 = prepare_data(9, take_first_elements = 100)

  0%|          | 0/2329 [00:00<?, ?it/s]

Computing probabilities


  0%|          | 0/1 [00:00<?, ?it/s]

Computing probabilities


  0%|          | 0/100 [00:00<?, ?it/s]

In [34]:
calculate_metrcis(0.0048, p_M1_YES_exg_9, p_M1_NO_exg_9)

(0.8514851485148515, 0.0625, 1.0, 0.11764705882352941)

In [36]:
#p_M1=list()
#for i in range(4,10):
#    p_M1.append(prepare_data(i, take_first_elements = 10000))

#calculate_metrcis(0.0047, p_M1[0][0], p_M1[0][1])
#calculate_metrcis(0.0048, p_M1[1][0], p_M1[1][1])